# Where should a drinks company run promotions?

## 📖 Background
Your company owns a chain of stores across Russia that sell a variety of alcoholic drinks. The company recently ran a wine promotion in Saint Petersburg that was very successful. Due to the cost to the business, it isn’t possible to run the promotion in all regions. The marketing team would like to target 10 other regions that have similar buying habits to Saint Petersburg where they would expect the promotion to be similarly successful.

### The data
The marketing team has sourced you with historical sales volumes per capita for several different drinks types.

- "year" - year (1998-2016)
- "region" - name of a federal subject of Russia. It could be oblast, republic, krai, autonomous okrug, federal city and a single autonomous oblast
- "wine" - sale of wine in litres by year per capita
- "beer" - sale of beer in litres by year per capita
- "vodka" - sale of vodka in litres by year per capita
- "champagne" - sale of champagne in litres by year per capita
- "brandy" - sale of brandy in litres by year per capita

## 💪 Competition challenge

1. Recommend 10 additional regions they should select for the promotion.
2. Tell the story that supports your recommendations.

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [2]:
# dataset
alcohol = pd.read_csv("data/alcohol-consumption-in-russia.csv")

# key variables
target_region = 'Saint Petersburg'
alcohol_categories = ['wine', 'beer', 'vodka', 'champagne', 'brandy']
years = sorted(alcohol['year'].unique())
print(len(years))

FileNotFoundError: [Errno 2] No such file or directory: 'data/alcohol-consumption-in-russia.csv'

# Preprocessing

In [3]:
# fill in nans
alcohol = alcohol.fillna(0)

# create total column
alcohol['total'] = alcohol[alcohol_categories].sum(axis = 1)

# filter out nulls
alcohol = alcohol[alcohol['total'] > 0]

NameError: name 'alcohol' is not defined

In [ ]:
alcohol[target_region] = alcohol['region'].apply(lambda x: 1 if x == target_region else 0)

In [ ]:
alcohol.describe()

# EDA

## What are the largest regions by alcohol category?

In [ ]:
def is_target_region(region):
    return region == target_region

In [ ]:
alcohol_category = 'beer'

df = alcohol.copy()

fig, axs = plt.subplots(7, 3, figsize = (16, 20))
axs = axs.flatten()

tmp = df.groupby(['year','region'])[alcohol_category].sum().reset_index()
tmp.columns = ['year', 'region', alcohol_category]
top_list = 10

i = 0
for year in years:
    tmp2 = tmp[tmp['year'] == year]
    tmp2 = tmp2.sort_values(by = alcohol_category, ascending = False)
    tmp2 = tmp2.head(top_list)
    tmp2[target_region] = tmp2['region'].apply(is_target_region) 
    sns.barplot(data = tmp2, x = alcohol_category, y = 'region', ax = axs[i], hue = target_region)
    axs[i].set_title(f"{year} Top-10 List {alcohol_category.capitalize()}")
    axs[i].legend().remove()
    i += 1
plt.tight_layout()

In [ ]:
df = alcohol[alcohol['region'] == target_region]
df = df.groupby(['year','region'])[alcohol_categories].sum().reset_index()
df = df.melt(id_vars=['year', 'region'])
df.columns = ['year', 'region', 'category', 'liters']
df = df.sort_values(by = 'liters', ascending = False)

fig, axs = plt.subplots(1, 2, figsize = (20, 4))
sns.lineplot(data = df, x = 'year', y = 'liters', hue = 'category', ax = axs[0], hue_order = alcohol_categories)
axs[0].set_title(f"{target_region} Alcohol Consumption Over Time")


df = alcohol.copy()
df = alcohol[alcohol['region'] == target_region].reset_index(drop = True)

for cat in alcohol_categories:
    df[f'pct_{cat}'] = df[cat] / df['total']

df = df.drop(alcohol_categories + ['total', target_region], axis = 1).fillna(0)

df = df.melt(id_vars = ['year', 'region'])
df.columns = ['year', 'region', 'category', 'pct']

plt.figure(figsize=(16, 6))
axs[1].set_title(f"{target_region} Alcohol consumption as a percent of total")
sns.lineplot(data = df, x = 'year', y = 'pct', hue = 'category', ax = axs[1])

plt.tight_layout()

# Feature Engineering

## Add historic features

In [ ]:
df = alcohol.copy()
most_recent_year = df['year'].max()
year_list = [most_recent_year - y for y in np.arange(1, 4)]
alcohol = df[['region','year'] + alcohol_categories]

In [ ]:
df = alcohol.copy()

data = []
tmp = df[df['year'] == most_recent_year]
data.append(tmp)

for year in year_list:
    tmp = df[df['year'] == year]
    data.append(tmp)

df = pd.pivot(pd.concat(data), values = alcohol_categories, index='region', columns = 'year')
df.columns = [x[0] + "_" + str(x[1]) for x in df.columns]

df = df.fillna(0)

## Find optimal group count for KMeans

In [ ]:
plt.figure(figsize = (16, 6))
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df)

ks = np.arange(1, 11)
wcss = []
for k in ks:
    model = KMeans(n_clusters = k, n_init = 'auto')
    model.fit(scaled_df)
    wcss.append(model.inertia_)

plt.plot(ks, wcss)
plt.xlabel("clusters")
plt.ylabel("inertia")
plt.tight_layout()

# Model Building

In [ ]:
n_clusters = 4
model = KMeans(n_clusters=n_clusters, n_init = 'auto')
labels = model.fit_predict(scaled_df)
df['group'] = labels + 1

# PCA

In [ ]:
pca = PCA(n_components=2)
pca_components = pca.fit_transform(scaled_df)

In [ ]:
tmp = pd.DataFrame(pca_components[:, :])
tmp.columns = ['PCA1', 'PCA2']
tmp['region'] = df.index
tmp['group'] = labels + 1
tmp

In [ ]:
group = tmp[tmp['region'] == target_region]['group'].values[0]

In [ ]:
plt.figure(figsize = (12, 8))
ax = sns.scatterplot(data = tmp, x = 'PCA1', y = 'PCA2', hue = 'group')

# Annotate each point in the scatter plot
for i in range(tmp.shape[0]):
    if tmp['group'][i] == group:
        ax.text(tmp['PCA1'][i] + 0.0,  # use proper indexing for DataFrame columns
                tmp['PCA2'][i] + 0.05,  # use proper indexing for DataFrame columns
                tmp['region'][i],  # use proper indexing for DataFrame columns
                horizontalalignment='left',
                size='small', color='black', weight=None)

plt.tight_layout()

In [ ]:
tmp = df[df['group'] == group].reset_index()
tmp = tmp.drop('group', axis = 1)
num_features = tmp.select_dtypes(exclude = 'object').columns

scaler = StandardScaler()
tmp2 = scaler.fit_transform(tmp[num_features])
tmp2 = pd.DataFrame(tmp2)
tmp2.columns = num_features
tmp2['region'] = tmp['region']

st_peters_vals = tmp2[tmp2['region'] == target_region][num_features].values
regions = [r for r in tmp['region'].unique() if r != target_region]

data = []
for r in regions:
    comp_vals = tmp[tmp['region'] == r][num_features].values
    similarity = euclidean_distances(st_peters_vals, comp_vals)
    item = {}
    item['region'] = r
    item['distance'] = similarity[0][0]
    data.append(item)

# Ten Regions with Similar Alcohol Consumption Habits to Saint Petersburg

In [ ]:
tmp = pd.DataFrame(data).sort_values(by = 'distance', ascending = True).reset_index(drop = True)
tmp.head(10)
similar_regions = tmp.head(10)['region'].values

# store all regions here
relevant_regions = np.append(similar_regions, target_region)

tmp.head(10)